In forward selection, variables are progressively incorporated into larger and larger subsets. The
algorithms start by training all possible single-variable machine learning models. Then, it selects
the feature that returned the best performing classifier or regression model. In the second step, it
creates machine learning models for all combinations of the features from the previous step with
all remaining variables in the data. It selects the pair of features that produce the best performing
algorithm. And it continues, adding 1 feature at a time to the feature subset from the previous step
until a stopping criteria is met.

The feature subsets are nested because they include the feature or features from the previous steps.
By progressively evaluating promising features, forward feature selection is more efficient than
exhaustive search.

Forward feature selection has the advantage that, by starting with smaller feature subsets, it is more
computationally efficient than other wrapper methods. However, for this same reason, it does not
contemplate feature interactions. Or at least not until sufficient features have been added to the
subset.

Forward feature selection needs a criteria to stop the search. The most obvious stopping condition
is when the performance of the classifier or regression model does not improve beyond a certain

threshold. This has the advantage of focusing the search on performance. On the downside, the
threshold for improvement is arbitrarily set by the user. Alternatively, we can stop the search after
a certain number of features have been selected.

In the coming paragraphs, we will implement forwardfeature selection with Scikit-learn and
MLXtend. The 2 Python implementations are very similar. Both offer a number of features as
stopping criteria. Scikit-learn also offers a threshold on performance improvement as a method to
stop the search.

# Import libraries

In [1]:
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression

# to select the features
from sklearn.feature_selection import SelectKBest, SelectPercentile
import pandas as pd
import csv
import seaborn as sns
sns.set_theme(style="whitegrid")
from tabulate import tabulate
import missingno as msno
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.io as pio
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score, mean_squared_error
import numpy as np

# Import data

In [2]:
file_path = '../train_1.csv'

data = pd.read_csv(file_path)
data

,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,53.622535,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.00
1,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,53.979870,...,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.00
2,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,53.656268,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.00
3,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,53.639405,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,22.00
4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,53.588771,...,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809,23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21188,106.957877,53.095769,82.515384,43.135565,1.177145,1.254119,146.88130,15.504479,65.764081,43.202659,...,3.555556,3.223710,3.519911,1.377820,0.913658,1,2.168889,0.433013,0.496904,2.44
21189,92.266740,49.021367,64.812662,32.867748,1.323287,1.571630,188.38390,7.353333,69.232655,50.148287,...,2.047619,2.168944,2.038991,1.594167,1.337246,1,0.904762,0.400000,0.212959,122.10
21190,99.663190,95.609104,99.433882,95.464320,0.690847,0.530198,13.51362,53.041104,6.756810,5.405448,...,4.800000,4.472136,4.781762,0.686962,0.450561,1,3.200000,0.500000,0.400000,1.98
21191,99.663190,97.095602,99.433882,96.901083,0.690847,0.640883,13.51362,31.115202,6.756810,6.249958,...,4.690000,4.472136,4.665819,0.686962,0.577601,1,2.210000,0.500000,0.462493,1.84


# Data preparation

In [4]:

X = data.iloc[:, 0:-1]
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0,
)

X_train.shape, X_test.shape

((14835, 80), (6358, 80))

# Forward feature selection

In [5]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [6]:
# step forward feature selection

sfs = SFS(
    estimator=RandomForestRegressor(n_estimators=5, random_state=0),
    k_features=15,  # the number of features to retain
    forward=True, # the direction of  the search
    verbose=1,  # print out intermediate steps
    scoring='r2',
    cv=3,
)

sfs = sfs.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:   45.8s finished
Features: 1/15[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  79 out of  79 | elapsed:  1.1min finished
Features: 2/15[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  78 out of  78 | elapsed:  1.6min finished
Features: 3/15[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  77 out of  77 | elapsed:  2.0min finished
Features: 4/15[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  76 out of  76 | elapsed:  2.3min finished
Features: 5/15[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  2.6min finished
Features: 6/15[Parallel(

In [7]:
sfs.k_feature_names_

('mean_atomic_mass',
 'wtd_gmean_atomic_mass',
 'range_atomic_mass',
 'wtd_std_atomic_mass',
 'range_fie',
 'entropy_atomic_radius',
 'entropy_Density',
 'range_Density',
 'std_ElectronAffinity',
 'mean_ThermalConductivity',
 'wtd_range_ThermalConductivity',
 'mean_Valence',
 'wtd_mean_Valence',
 'gmean_Valence',
 'entropy_Valence')

In [8]:
X_train_t = sfs.transform(X_train)
X_test_t = sfs.transform(X_test)

X_test_t

array([[ 76.5177175 ,  65.41137811, 122.90607   , ...,   2.4625    ,
          2.21336384,   1.36892236],
       [ 67.4025    ,  66.40422577,  23.115     , ...,   4.5       ,
          4.24264069,   0.63651417],
       [ 89.33718   ,  69.13330879, 124.90825   , ...,   2.17142857,
          2.49146188,   1.56495725],
       ...,
       [ 90.30468   ,  64.31362755, 128.2426    , ...,   2.465     ,
          2.49146188,   1.56495725],
       [ 63.45766667,  66.73940342,  31.093     , ...,   5.24675325,
          3.63424119,   1.01140426],
       [ 69.17125   ,  35.42963674, 121.3276    , ...,   2.07103394,
          2.16894354,   1.5941667 ]])

In [9]:
X_train_t.shape, X_test_t.shape

((14835, 15), (6358, 15))

# Scaling

In [11]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_sc= min_max_scaler.fit_transform(X_train_t)

X_test_sc=min_max_scaler.fit_transform(X_test_t)

In [12]:
scaler = preprocessing.MinMaxScaler()
y_train_sc = scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_sc = scaler.fit_transform(y_test.values.reshape(-1, 1))

# Baseline model

In [13]:
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.dummy import DummyRegressor

In [14]:
from sklearn.metrics import (
    mean_absolute_error as mae,
    r2_score as r2,
    mean_absolute_percentage_error as mape,
    mean_squared_error as mse)

In [15]:
def report_metrics(model, X_train_sc, X_test_sc, y_train_sc, y_test_sc, label):
    print(f'Train MAE ({label}):', round(mae(y_train_sc, model.predict(X_train_sc)), 4))
    print(f'Test MAE ({label}) :', round(mae(y_test_sc, model.predict(X_test_sc)), 4), '\n')

    print(f'Train R^2 ({label}):', round(r2(y_train_sc, model.predict(X_train_sc)), 4))
    print(f'Test R^2 ({label}) :', round(r2(y_test_sc, model.predict(X_test_sc)), 4), '\n')

In [16]:
LR_baseline = LinearRegression().fit(X_train_sc, y_train_sc)
report_metrics(LR_baseline, X_train_sc, X_test_sc, y_train_sc, y_test_sc, 'LR_baseline')

Train MAE (LR_baseline): 0.1261
Test MAE (LR_baseline) : 0.1087 

Train R^2 (LR_baseline): 0.5744
Test R^2 (LR_baseline) : 0.4509 



# Cross-Validation

In [18]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor

In [19]:
# Assuming you have defined X_train_sc and y_train_sc

list_of_models = [
    RandomForestRegressor(),
    XGBRegressor(),
    KNeighborsRegressor(),
    LGBMRegressor(),
    SVR(),
    DecisionTreeRegressor(),
    GradientBoostingRegressor()
]

list_of_model_names = [type(x).__name__ for x in list_of_models]
cv_results = pd.DataFrame(
    data=0.0,
    index=list_of_model_names,
    columns=['fit_time', 'score_time', 'neg_mean_squared_error', 'neg_mean_absolute_error'])

for model in list_of_models:
    cv_result = cross_validate(
        estimator=model,
        X=X_train_sc,
        y=y_train_sc,
        scoring=['neg_mean_squared_error', 'neg_mean_absolute_error'],
        cv=30,
        n_jobs=-1)

    cv_results.loc[type(model).__name__] = [
        np.mean(cv_result['fit_time']),
        np.mean(cv_result['score_time']),
        np.mean(cv_result['test_neg_mean_squared_error']),
        np.mean(cv_result['test_neg_mean_absolute_error'])  # Removed the negative sign here
    ]

print(cv_results)

                            fit_time  score_time  neg_mean_squared_error  \
RandomForestRegressor      45.892902    0.066034               -0.004685   
XGBRegressor                7.670053    0.007209               -0.005140   
KNeighborsRegressor         0.092373    0.048569               -0.006610   
LGBMRegressor               0.816087    0.010529               -0.005852   
SVR                        11.068261    0.818087               -0.012106   
DecisionTreeRegressor       0.517493    0.002578               -0.006928   
GradientBoostingRegressor  14.625863    0.004393               -0.009047   

                           neg_mean_absolute_error  
RandomForestRegressor                    -0.038035  
XGBRegressor                             -0.043551  
KNeighborsRegressor                      -0.045641  
LGBMRegressor                            -0.049047  
SVR                                      -0.080473  
DecisionTreeRegressor                    -0.043362  
GradientBoostingRegr

In [20]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_sc, y_train_sc)

# Make predictions on the test set
y_pred = model.predict(X_test_sc)

# Evaluate the model's performance
mse = mean_squared_error(y_test_sc, y_pred)
mae = mean_absolute_error(y_test_sc, y_pred)
r2 = r2_score(y_test_sc, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)

C:\Users\Margarita\AppData\Local\Temp\ipykernel_24488\377911579.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_sc, y_train_sc)


Mean Squared Error (MSE): 0.010474788203692785
Mean Absolute Error (MAE): 0.0670117528082491
R-squared (R2): 0.6964966640866743


In [21]:
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train_sc, y_train_sc)

# Make predictions on the test set
y_pred = model.predict(X_test_sc)

# Evaluate the model's performance
mse = mean_squared_error(y_test_sc, y_pred)
mae = mean_absolute_error(y_test_sc, y_pred)
r2 = r2_score(y_test_sc, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)

Mean Squared Error (MSE): 0.009969758316757437
Mean Absolute Error (MAE): 0.06784181844209433
R-squared (R2): 0.7111297289697209
